In [ ]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import load_model
from keras.engine.topology import Layer
from keras.datasets import mnist

In [ ]:
batchsize = 64
n_z = 20

In [ ]:
def encoder(inputs):

    encode = Conv2D(32, (2, 2), strides=2)(inputs)

    encode = MaxPooling2D((2, 2))(encode)

    encode = Activation('relu')(encode)

    encode = Conv2D(32, (2, 2), strides=2)(encode)

    encode = MaxPooling2D((2, 2))(encode)

    encode = Activation('relu')(encode)

    encode = Flatten()(encode)

    encode = Dense(512, activation = 'relu')(encode)
    
    encode = Dropout(0.5)(encode)

    mu = Dense(n_z, activation = 'linear')(encode)

    log_sigma = Dense(n_z, activation = 'linear')(encode)
    
    return mu, log_sigma

In [ ]:
def decoder(latent_vector): 
    
    decode = Dense(512, activation = 'relu')(latent_vector)
    
    decode = Reshape((14, 14, 1))(decode)
    
    decode = Conv2D(32, (2, 2), strides=2)(decode)
    
    decode = UpSampling2D((2, 2))(decode)
    
    decode = Activation('relu')(decode)
    
    decode = Conv2D(32, (2, 2), strides=2)(decode)
    
    decode = UpSampling2D((2, 2))(decode)
    
    decode = Activation('relu')(decode)
    
    out = Conv2D(1, (2, 2), activation = 'sigmoid')(decode)
    
    mu_out = Dense(n_z, activation = 'linear')(out)

    log_sigma_out = Dense(n_z, activation = 'linear')(out)
    
    return mu_out, log_sigma_out, out

In [ ]:
def loss():
    # p_z = gaussian(0,1)
    # q_z_x = gaussian(mean_y, e^ln_var_y) - check this exp
    # p_x_z = gaissian(mean_x, e^ln_var_x)
    # L = F.gaussian_kl_dvergance(q_z_x, p_z) - F.gaussian_nll(p_x_z)
        
    #? What about input x?
        
    # Why use log variance instead of variance:

In [ ]:
def sample_z(mu, log_sigma):
    eps = K.random_normal(shape=(mu, n_z), mean=0., stddev=1.)
    return mu + K.exp(log_sigma / 2) * eps

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
batches = int(len(x_train)/batchsize)

In [ ]:
type(x_train)

In [ ]:
x_train.shape

In [ ]:
def batch_generator():
    
    while True:

        for batch in range(batches):
            x = []
            
            data = x_train[batch * batchsize: (batch + 1) * batchsize]
            for i in data:
                x.append(np.reshape(i, (28, 28, 1)))
            
            x_train1 = np.asarray(x)
            yield (x_train1, x_train1) # the data is also the target (since we're re-generating images)

In [ ]:
inputs1 = Input(shape = (28, 28, 1))

mu, log_sigma = encoder(inputs1)

latent_vector = sample_z(mu, log_sigma)

mu_out, log_sigma_out, out_img = decoder(latent_vector)

model = Model(inputs = input1, outputs = out_img)

In [ ]:
model.fit_generator(batch_generator(), steps_per_epoch = batches, epochs=10, shuffle = True, callbacks = callback_list)